# ESO Phase 3 for PHANGS <a class="tocSkip">
    
The details are outlined [here](https://www.eso.org/sci/observing/phase3.html)

* 1. [Register](https://www.eso.org/sci/observing/phase3/rm.html) your Phase 3 submission
* 2. Preparing your data.
  
      - List of Keywords for header (FS)
      - Filename (<68 characters), unique name across all folders
      - mosaicked images require a weight map image to be associated$^1$
      - optional: preview images, graphics, reports from DR (must be referenced in main SDP header)
      - should include an exposure map (EXPTIME set to median of non 0 pixels), unless standard offset pattern with constant integration time is used.

      * **for IFU Cubes do the following things**:
      * astrometric calibration
      * calibration of the dispersion axis to physical wavelength scale
      * removal/correction for instrumental and sky background signal (if applicable)
      * calibration of the detected signal to physical scale (spectral flux density)
      * re-sampling to a regular 3-dimensional grid 
      * signal combination of multiple exposures (if applicable)
      * error propagation in each processing step to obtain a final error estimate for the science data
      * propagation of pixel quality information

$^1$ statistical significance of each pixel in terms of a number that is proportional to the inverse variance of the background signal, i.e. not including the Poisson noise of sources. This additional data array is often called weight map or confidence map. It has the same dimensions as the image array and can be submitted as an associated data product with product category declared in the header of the science file as follows:      

**IFU 3D Cubes Page 1**: Storing several sets of HDU's with associated science data, error and data quality in a single FITS file, though permitted in [3], is not supported by the ESO/SDP
standard. Only three HDU’s are permitted of which one must be the science data and spectral flux density (physical units declared in `BUNIT`)

* 3. Uploading your data to ESO: Press `CLOSE` and hope for the best.

* 4. Verifying compliance with data format requirements

* 5. Uploading the data [release description](https://www.eso.org/sci/observing/phase3/release_description.html)

     https://www.eso.org/sci/observing/phase3/release-description-tmpl.doc

* 6. Finalizing your data submission








## Preparation
 
### Load Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# some basic packages
import os                 # filesystem related stuff
import json
from pathlib import Path  # use instead of os.path and glob
import sys                # mostly replaced by pathlib
import re

import errno      # more detailed error messages
import warnings   # handles warnings
import logging    # use logging instead of print

from collections import OrderedDict  

# packages for scientific computing
import numpy as np
import scipy as sp

# packages for creating plots and figures
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# special functions for astronomy 
from astropy.table import Table  # useful datastructure
from astropy.table import vstack # combine multiple tables

from astropy.io import fits      # open fits files
from astropy.io import ascii     # handle normal files

from astropy.wcs import WCS               # handle coordinates
from astropy.coordinates import SkyCoord  # convert pixel to sky coordinates
from astropy.visualization import simple_norm

from astropy.stats import sigma_clipped_stats  # calcualte statistics of images

import astropy.units as u        # handle units
from astropy.time import Time

from spectral_cube import SpectralCube

tab10 = ['#e15759','#4e79a7','#f28e2b','#76b7b2','#59a14e','#edc949','#b07aa2','#ff9da7','#9c755f','#bab0ac']    

In [ ]:
# first we need to specify the path to the raw data
data_raw = Path('g:\Archive')
basedir = Path('..')

## Headers of Raw files

In [ ]:
import pickle
from astropy.time import Time

In [ ]:
with open(basedir / 'data' / 'ESO' / 'headers_P01_NGC0628.pkl', 'rb') as f:
    headers = pickle.load(f)

In [ ]:
print('\n'.join(list(headers.keys())))

In [ ]:
header = headers['Raw/M.MUSE.2014-12-08T12:30:51.293.fits']

In [ ]:
minimum = ('key',Time('2020-02-01'))

for filename in headers.keys():

    header = headers[filename]
    for key in header.keys():
        if 'MJD-OBS' in header[key].keys():
            date = Time(header[key]['MJD-OBS'],format='mjd')
            if date < minimum[1]:
                print(date.iso)
                minimum = (filename,date)

In [ ]:
headers['Raw/M.MUSE.2014-12-08T12:30:51.293.fits']['LINES']

## GASP Files

It might be useful to look at [existing data realeases](https://www.eso.org/rm/publicAccess#/dataReleases) like the [GASP DR1](https://www.eso.org/sci/publications/announcements/sciann17080.html)

In [ ]:
from astropy.io.fits.header import Card, Header

In [ ]:
with open(basedir / 'data' / 'ESO' / 'GASPheader.txt') as f:
    raw = f.read()

extensions = raw.split('\nEND')

In [ ]:
Cards_raw = [[x.strip() for x in re.split('=| / ',line)] for line in extensions[0].split('\n')]

for card in Cards_raw:
    
    if card[1] == 'T':
        card[1] = True
        continue
    elif card[1] == 'F':
        card[1] = False
        continue
    
    try:
        i = int(card[1])
        card[1] = i
        continue
    except:
        pass
    try:
        f = float(card[1])
        card[1] = f
        continue
    except:
        pass
        
    
    card[1] = card[1].strip("'").strip(' ')
    
header = Header([Card(*card) for card in Cards_raw])

In [ ]:
with fits.open(data_raw / 'MUSE' / 'GASP' / 'ADP.2017-10-20T13_29_03.027.fits') as hdul:
    header = hdul[0].header

In [ ]:
header

In [ ]:
keywords = ascii.read(basedir / '..' / 'PHANGS' / 'ESO' / 'keywords.csv',delimiter=';')

In [ ]:
with fits.open(data_raw / 'MUSE' / 'MOSAIC' / 'NGC628' / 'NGC628_DATACUBE_FINAL.fits') as hdul:
    header2 = hdul[0].header

In [ ]:
adopted = ['ORIGIN',
 'TELESCOP',
 'INSTRUME',
 'FILTER',
 'OBJECT',
 'EQUINOX',
 'RADESYS',
 'EXPTIME',
 'TEXPTIME',
 'MJD-OBS',
 'MJD-END',
 'PROG_ID',
 'OBIDi',
 'NCOMBINE',
 'OBSTECH',
 'REFERENC']

In [ ]:
header2

In [ ]:
Time(57280.20875754,format='mjd').iso

In [ ]:
Time(header['DATE-OBS']).iso

In [ ]:
Time(header['MJD-OBS'],format='mjd').iso

## Display Path tree

In [ ]:
class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))
    
paths = DisplayablePath.make_tree(Path('..'))
for path in paths:
    print(path.displayable())

## Open datacubes

In [ ]:
data_raw = Path('g:\Archive')
name = 'NGC1087'
filename = data_raw / 'MUSE' / 'MOSAIC' / name / f'{name}_DATACUBE_FINAL.fits'

with fits.open(filename , memmap=True, mode='denywrite') as hdul:
    cube=SpectralCube(data=hdul[1].data,wcs=WCS(hdul[1].header))

In [ ]:
plt.plot(cube[:,0,0]) 

In [ ]:
cube = SpectralCube.read(filename,format='fits',hdu=1)

In [ ]:
cube

## Appendix



```
SIMPLE = T / file does conform to FITS standard
BITPIX = 8 / number of bits per data pixel
NAXIS = 0 / number of data axes
EXTEND = T / FITS dataset may contain extensions
DATE = '2015-05-20T10:20:35' / file creation date (YYYY-MM-DDThh:mm:ss UT)
ORIGIN = 'ESO-PARANAL' / European Southern Observatory
TELESCOP= 'ESO-VLT-U4' / ESO Telescope
INSTRUME= 'MUSE ' / ESO Instrument name
RA = 183.46028 / [deg] Image centre (J2000.0)
DEC = 7.20120 / [deg] Image centre (J2000.0)
EQUINOX = 2000. / Standard FK5
RADECSYS= 'FK5 ' / Coordinate system
EXPTIME = 2520.0 / Total integration time per pixel
TEXPTIME= 2520.0 / Total integration time all exposures
NCOMBINE= 3 / # of combined raw science data files
MJD-OBS = 57126.04953770 / 2015-04-14T01:11:20.1
MJD-END = 57126.08556889 / 2015-04-14T02:03:13.2
DATE-OBS= '2015-04-14T01:11:20.057' / Observing date
OBJECT = 'NGC 4191' / Target designation
OBID1 = 1164690 / Observation block ID
PROG_ID = '095.B-0686(A)' / ESO programme identification code
PROV1 = 'MUSE.2015-04-14T01:11:20.057.fits' / Original science file
PROV2 = 'MUSE.2015-04-14T01:27:10.371.fits' / Original science file
PROV3 = 'MUSE.2015-04-14T01:49:13.152.fits' / Original science file
OBSTECH = 'IFU ' / Technique of observation
PRODCATG= 'SCIENCE.CUBE.IFS' / Data product category
ASSON1 = 'IMAGE_FOV_0001.fits' / Collapsed data cube
ASSOC1 = 'ANCILLARY.IMAGE' / Category of associated file
WAVELMIN= 475.0 / [nm] Minimum wavelength
WAVELMAX= 843.0 / [nm] Maximum wavelength
SPEC_RES= 2500 / Spectral resolving power at central wavelength
SKY_RES = 0.94 / [arcsec] FWHM effective spatial resolution (mea
SKY_RERR= 0.10 / [arcsec] Error of SKY_RES (estimated)
ABMAGLIM= 22.5 / 5-sigma magnitude limit for point sources
PIXNOISE= 4.50E-20 / [erg/s/cm**2/Angstrom] pixel-to-pixel noise
FLUXCAL = 'ABSOLUTE' / Certifies the validity of BUNIT
PROCSOFT= 'muse/1.0.4' / Data reduction software/version no.
REFERENC= ' ' / Bibliographic reference
CHECKSUM= 'RGhHRGhGRGhGRGhG' / HDU checksum updated 2015-07-15T16:27:36
DATASUM = ' 0' / data unit checksum updated 2015-05-20T10:20:48
END

						Extension 1

XTENSION= 'IMAGE ' / IMAGE extension
BITPIX = -32 / number of bits per data pixel
NAXIS = 3 / number of data axes
NAXIS1 = 329 / length of data axis 1
NAXIS2 = 317 / length of data axis 2
NAXIS3 = 3681 / length of data axis 3
NAXIS1 = 329 / length of data axis 1
NAXIS2 = 317 / length of data axis 2
NAXIS3 = 3681 / length of data axis 3
PCOUNT = 0 / required keyword; must = 0
GCOUNT = 1 / required keyword; must = 1
EXTNAME = 'DATA ' / This extension contains data values
HDUCLASS= 'ESO ' / class name (ESO format)
HDUDOC = 'DICD ' / document with class description
HDUVERS = 'DICD version 6' / version number (according to spec v2.5.1)
HDUCLAS1= 'IMAGE ' / Image data format
HDUCLAS2= 'DATA ' / this extension contains the data itself
ERRDATA = 'STAT ' / pointer to the variance extension
OBJECT = 'NGC 4191 (DATA)'
BUNIT = '10**(-20)*erg/s/cm**2/Angstrom'
CRPIX1 = 170.061496799859 / Pixel coordinate of reference point
CRPIX2 = 152.429853570976 / Pixel coordinate of reference point
CD1_1 = -5.55555555555556E-05 / Coordinate transformation matrix element
CD1_2 = 0. / Coordinate transformation matrix element
CD2_1 = 0. / Coordinate transformation matrix element
CD2_2 = 5.55555555555556E-05 / Coordinate transformation matrix element
CUNIT1 = 'deg ' / Units of coordinate increment and value
CUNIT2 = 'deg ' / Units of coordinate increment and value
CTYPE1 = 'RA---TAN' / Right ascension, gnomonic projection
CTYPE2 = 'DEC--TAN' / Declination, gnomonic projection
CSYER1 = 1.66499066997E-05 / [deg] Systematic error in coordinate
CSYER2 = 6.60827614552E-06 / [deg] Systematic error in coordinate
CRVAL1 = 183.46
CRVAL2 = 7.20083
CTYPE3 = 'AWAV '
CUNIT3 = 'Angstrom'
CD3_3 = 1.25
CRPIX3 = 1.
CRVAL3 = 4749.81640625
CD1_3 = 0.
CD2_3 = 0.
CD3_1 = 0.
CD3_2 = 0.
CRDER3 = 0.026 / [Angstrom] Random error in spectral coordinate
CHECKSUM= 'ZUJFZS9DZSGDZS9D' / HDU checksum updated 2015-07-15T16:27:36
DATASUM = '39318882' / data unit checksum updated 2015-05-20T10:20:54
END

								Extension 2

XTENSION= 'IMAGE ' / IMAGE extension
BITPIX = -32 / number of bits per data pixel
NAXIS = 3 / number of data axes
NAXIS1 = 329 / length of data axis 1
NAXIS2 = 317 / length of data axis 2
NAXIS3 = 3681 / length of data axis 3
NAXIS1 = 329 / length of data axis 1
NAXIS2 = 317 / length of data axis 2
NAXIS3 = 3681 / length of data axis 3
PCOUNT = 0 / required keyword; must = 0
GCOUNT = 1 / required keyword; must = 1
EXTNAME = 'STAT ' / This extension contains data variance
HDUCLASS= 'ESO ' / class name (ESO format)
HDUDOC = 'DICD ' / document with class description
HDUVERS = 'DICD version 6' / version number (according to spec v2.5.1)
HDUCLAS1= 'IMAGE ' / Image data format
HDUCLAS2= 'ERROR ' / this extension contains variance
HDUCLAS3= 'MSE ' / the extension contains variances (sigma**2)
SCIDATA = 'DATA ' / pointer to the data extension
OBJECT = 'NGC 4191 (STAT)'
BUNIT = '(10**(-20)*erg/s/cm**2/Angstrom)**2'
CRPIX1 = 170.061496799859 / Pixel coordinate of reference point
CRPIX2 = 152.429853570976 / Pixel coordinate of reference point
CD1_1 = -5.55555555555556E-05 / Coordinate transformation matrix element
CD1_2 = 0. / Coordinate transformation matrix element
CD2_1 = 0. / Coordinate transformation matrix element
CD2_2 = 5.55555555555556E-05 / Coordinate transformation matrix element
CUNIT1 = 'deg ' / Units of coordinate increment and value
CUNIT2 = 'deg ' / Units of coordinate increment and value
CTYPE1 = 'RA---TAN' / Right ascension, gnomonic projection
CTYPE2 = 'DEC--TAN' / Declination, gnomonic projection
CSYER1 = 1.66499066997E-05 / [deg] Systematic error in coordinate
CSYER2 = 6.60827614552E-06 / [deg] Systematic error in coordinate
CRVAL1 = 183.46
CRVAL2 = 7.20083
CTYPE3 = 'AWAV '
CUNIT3 = 'Angstrom'
CD3_3 = 1.25
CRPIX3 = 1.
CRVAL3 = 4749.81640625
CD1_3 = 0.
CD2_3 = 0.
CD3_1 = 0.
CD3_2 = 0.
CHECKSUM= 'JfcaJeZYJeaaJeYW' / HDU checksum updated 2015-05-20T10:20:55
DATASUM = '2131780454' / data unit checksum updated 2015-05-20T10:20:55
END


```

